- Andrea De Vita
- Enrico Lupi
- Manfredi Miranda
- Francesco Zane

-----------------------

# Streaming Processing of the QUAX Experiment Data for the Detection of Galactic Axions

## Abstract

The axion is a hypothetical particle introduced to solve the strong CP problem of Quantum Chromo Dynamics. It is speculated that axions may also constitute the dark matter (DM) content in our galaxy. The [QUAX](https://www.pd.infn.it/eng/quax/) (QUaerere AXions) experiment aims at detecting this particle by using a copper cavity immersed in a static magnetic field of 8.1 T, cooled down at a working temperature of about 150 mK.

The goal of this project is to create a quasi real-time processing chain of the data produced by the QUAX experimental apparatus and a live monitoring system of the detector data, using [Apache Kafka](https://kafka.apache.org/) and [Apache Spark](https://spark.apache.org/).

## Table of Contents

1. [Introduction](#introduction) <br>
    1.1. [Experiment](#intro_experiment) <br>
    1.2. [Data Structure](#intro_data_structure) <br>
    1.3. [Cluster](#intro_cluster) <br>
2. [Data Processing](#processing) <br>
    2.1. [Pipeline Overview](#pipeline) <br>
    2.2. [Kafka - Streaming Data](#kafka) <br> 
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2.1. [Kafka Topics](#kafka_topic) <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2.2. [Data Pre-processing](#kafka_preprocessing) <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2.3. [Producer](#kafka_producer) <br>
    2.3. [Spark - Distributed Processing](#spark) <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3.1. [Spark Structured Streaming](#spark_streaming) <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3.2. [FFT and Averaging](#spark_fft) <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3.3. [Output Message](#spark_output) <br>
    2.4. [Live Plot and Monitoring](#live_plot) <br>
3. [Performance](#performance) <br>
    3.1. [Kafka](#test_kafka) <br>
    3.2. [Spark](#test_spark) <br>
4. [Conclusion](#conclusion)

## 1. Introduction <a name="introduction"></a>

### 1.1. Experiment <a name="intro_experiment"></a>


![quax lab](Images\labQuax.webp)

The QUAX experiment aims at axion detection by using a copper cavity immersed in a static magnetic field of 8.1T, cooled down at a working temperature of about 150mK. The axion is expected to couple with the spin of the electron, interacting with the cavity and inducing a radio-frequency that can be sensed via a Josephson parametric amplifier. For a given configuration of the RF cavity, a scan of the phase of the electromagnetic field is performed to be able to possibly identify a localised excess, a hint of the coupling of an axion with the photon. 

The data acquisition system of the QUAX experiment generates two streams of digitized reading of the amplifiers, representing the real and imaginary components of the measured phase. To improve the signal over noise ratio, a QUAX data-taking run extends over a long time (up to weeks), repeating the scans over multiple times. Data are saved locally on the DAQ servers in the form of binary files, each corresponding to a multitude of continuous scans performed in the entire frequency range. A single pair of raw files is thus representative of only a few seconds of data taking, but are already including several (thousands) scans. 

### 1.2. Data Structure <a name="intro_data_structure"></a>

The dataset is composed of 2 sets (named duck_i and duck_q respectively) of .dat binary files, each one comprised of a continuous series of ADC readings from the amplifier. Each ADC reading is written in the raw files as a 32 bit floating point value. The ADC readout frequency is 2 × 10<sup>6</sup> Hz (2 MegaSample per second, or 2MS/s), thus resulting in a raw data throughput of 128 Mbps (16 MB/s). During data taking the readouts are formatted in .dat file such that each file is comprised of 8193 × 2<sup>10</sup> samples. This results in producing a pair of .dat files (duck_i and duck_q) every 4.2 s.

The dataset is provided on a cloud storage s3 bucket hosted on Cloud Veneto.

### 1.3. Cluster <a name="intro_cluster"></a>

This project has been done on a cluster composed by 4 virtual machines, each with 4 VCPUs with 25 GB disk space and 8 GB RAM each. The virtual machines are hosted on [CloudVeneto](https://cloudveneto.it/), an OpenStack-based cloud managed by University of Padova and INFN. Spark version 3.3.2 (using Scala version 2.12.15) and Kafka version 3.4.0 will be used.

## 2. Data Processing <a name="processing"></a>

The processing of the raw data is comprised of two phases:
1. Run a Fourier transform on each scan to move from the time domain to the frequency domain
2. Average (in bins of frequency) all scans in a data-taking run, to extract a single frequency scan
 
This procedure is highly parallelizable, and should be implemented in a quasi-online pipeline for two main reasons:
1. Monitoring the scans during the data taking to promptly spot and identify possible issues in the detector setup or instabilities in the condition of the experiment
2. Data is continuously produced with a very large rate, and the local storage provided by the DAQ server of the QUAX experiment is not really suited for large-volume and long-lasting datasets

### 2.1. Pipeline Overview <a name="pipeline"></a>

The data processing pipeline will be implemented as follows:
- Each pair of files is unpacked according to their schema and split into scans.
- Data is produced to a Kafka topic by a stream-emulator script every 5 seconds to simulate the fixed ADC scanning rate and the fixed size of files written to disk. 
- The processing of each file runs is performed in a distributed framework using pySpark: for each scan, a FFT is executed in parallel and the results of all FFTs are averaged.
- The results are re-injected into a new Kafka topic hosted on the same brokers.
- A final consumer performs the plotting, displaying live updates of the scans and continuously updating the entire "run-wide" scan using bokeh.

The overall pipeline can be thus summarised as:
![pipeline schema](Images\Pipeline_Schema2.png)

### 2.2. Kafka - Streaming Data <a name="kafka"></a>

Apache Kafka is an open-source distributed event streaming platform for high-performance data pipelines, streaming analytics, data integration, and mission-critical applications. As discussed previously, in this work it will be used to handle the live streaming of data from the DAQ servers all the way to the final live plot.

#### 2.2.1. Kafka Topics <a name="kafka_topic"></a>

The first step is to create a topic on the broker to hold the data from the DAQ. We create it with 4 separate partitions and no replication. The meaning of the name *chunk_data* will be made clear in the next section.

We will also create a second topic for later, aptly named *results*, where to publish the results of the data processing, i.e. the FFT and averaging.

In [ ]:
# connect to the cluster to run admin functions
kafka_admin = KafkaAdminClient(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
)

# define new topic to hold data
topic_in = NewTopic(name='chunk_data',
                    num_partitions=4, 
                    replication_factor=1)

In [ ]:
topic_out = NewTopic(name='results',
                     num_partitions=4, 
                     replication_factor=1)

#### 2.2.2. Data Preprocessing <a name="kafka_preprocessing"></a>

The size of the files produced by the DAQ is 32 MB, which means that each message handled by Kafka should be 64 MB as we need both the real and imaginary componenets. Unfortunately, the default message size in Kafka is only 1 MB. There are of course ways to circumvent this limit, namely:
 
- at the broker level, changing the *replica.fetch.max.bytes* in the broker settings and increasing the *max.message.bytes* for the topic to the desired value
- at the consumer level, increasing the *max.partition.fetch.bytes*, otherwise the consumer will fail to fetch these messages and will get stuck on processing
- at the producer level, increasing the *max.request.size* to ensure large messages can be sent

While this solution is possible, it is still against the philosophy of Kafka: sending large messages is considered inefficient as they should be huge in number but not in size.

We thus decided to first unpack the data into slices and send a pair of real and imaginary slices as a message. Since for each FFT we want *n<sub>bins</sub>* = 3 × 2<sup>10</sup> = 3072 bins and we have a total of 8193 × 2<sup>10</sup> samples per file, the amount of slices to compute FFTs on for each file (and thus of mesages to be sent) is

$$n_{slices} = \cfrac{n_{samples}}{n_{bins}} = \cfrac{8193 \times 2^{10} }{3 \times 2^{10}} = 2731$$

The code is actually more flexible, and allows us to send messages with an arbitrary number of slices inside (the maximum being 40, to have messages smaller than 1 MB). For simplicity, we will still only send one slice per message; a more in-depth discussion is present in the [performance tests](#test_kafka) section.

In [ ]:
slices_per_msg = 1
msg_number = math.ceil(n_slice/slices_per_msg)

# read all data from input files
real = bytearray(binary_data_real)
imag = bytearray(binary_data_imm)

# unpack data
for f in range(msg_number):
    start = 4*n_bins*slices_per_msg*f
    end = 4*n_bins*slices_per_msg*(f+1)
    if end > 4*n_samples:
        end = 4*n_samples
        
    r_bin = real[start:end]
    i_bin = imag[start:end]
    msg = r_bin + i_bin

#### 2.2.3. Producer <a name="kafka_producer"></a>

Lastly we can initialize the Kafka producer, the one responsible to read data from the files and actually sending it to the correct topic. The message, as descibed before, is given by two consecutives byte arrays containing the real and imaginary slices. The key, instead, contains the number of the file and of the particular slice contained in the message.

In [ ]:
# Create a Kafka producer instance
chunk_producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

def send_chunks(file_paths,dirPath,DAQ_period=5):
    
    partners = sorted(find_partner(file_paths), key=lambda x: get_number_from_filename(x[0]))
    
    for couple in partners: 
        start_time = time.time()
            
        couple=[dirPath+x for x in couple]
        binary_data_real = read_binary_file(couple[0])
        binary_data_imm = read_binary_file(couple[1])

        real = bytearray(binary_data_real)
        imag = bytearray(binary_data_imm)
        
        file_num=int(couple[0][-9:-4])
        
        for f in range(msg_number):
            start = 4*n_bins*slices_per_msg*f
            end = 4*n_bins*slices_per_msg*(f+1)
            if end > 4*n_samples:
                end = 4*n_samples
            r_bin = real[start:end] # one float every 4 bytes
            i_bin = imag[start:end]
            msg = r_bin + i_bin
        
            # key = file + bin number
            key = (file_num).to_bytes(2, "big") + f.to_bytes(2, "big")
           
            #print("Sending file",file_num,"\tslice number:",f+1,end="\r")
            chunk_producer.send(topic = "chunk_data",
                                key   = key,
                                value = msg)
            
        chunk_producer.flush()  # Flush the producer buffer
        
        end_time = time.time()
        deltat = end_time - start_time
        
        print("                                                                 ")
        print("File", file_num,"completed in ", deltat, " sec!")
        print("------------------------------")
        
        if deltat < DAQ_period:
            time.sleep(DAQ_period - deltat)
        
send_chunks(file_paths,folder_path)

### 2.3. Spark - Distributed Processing <a name="spark"></a>

Apache Spark is an open-source unified analytics engine for large-scale data processing. As outlines in the overview, Spark will do the "heavy lifting" of the data processing by computing the FFTs for each slice in parallel and averaging them.

The first step is to create a spark application. Note that the default number of shuffle partitions has been set to 4.

In [ ]:
spark = SparkSession.builder \
        .master("spark://10.67.22.8:7077") \
        .appName("Spark structured streaming application") \
        .config("spark.executor.memory", "512m") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .config("spark.sql.adaptive.enabled", "false") \
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false") \
        .config("spark.sql.shuffle.partitions", 4) \
        .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2") \
        .getOrCreate()

#### 2.3.1 Spark Structured Streaming <a name="spark_streaming"></a>

In order to deal with the quasi-continuous stream of data from Kafka we will use Spark Structured Streaming. This lets us use the DataFrame API and consider the incoming data as new rows of an unbound table.

Given the simplicity and lack of multiple features of the data we could also have chosen to use Spark Streaming, which works by dividing the input data into a sequence micro-batches (DStream) that can be treated as static datasets. Unfortunately, Spark Streaming is considered deprecated, and as such the packages necessary to connect it to Kafka are not present in Spark version 3, which we are currently using.

We first create an input (streaming) DataFrame subscribed to the *chunk_data* topic.

In [ ]:
inputDF = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("kafkaConsumer.pollTimeoutMs", 30_000) \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", n_slice) \
    .option("subscribe", "chunk_data") \
    .load()

Let's explain what some of these options are:
- *startingOffset* refers to the start point when a query is started: either "earliest" which is from the earliest offsets, "latest" which is just from the latest offsets, or a json string specifying a starting offset for each TopicPartition.
- *kafkaConsumer.pollTimeoutMs* is the timeout in milliseconds to poll data from Kafka in executors.
- *maxOffsetsPerTrigger* is the rate limit on maximum number of offsets processed per trigger interval. The specified total number of offsets will be proportionally split across topicPartitions of different volume.

Note that *maxOffsetsPerTrigger* is equal to *n<sub>slices</sub>* (2731), which means that we are never going to deal with batches bigger than a single file. This does not mean, however, that the content of each file will always be processed together, as it is still possible to have batches of smaller size. This last case is not a problem, though, as the division in files depends only on the DAQ system and does not reflect any underlying physics.

The input DataFrame from Kafka has the following schema:

In [ ]:
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

#### 2.3.2 FFT and Averaging <a name="spark_fft"></a>

We are only interested in the *key* and *value* pair. We then unpack the *value* column into a list of floats and compute the Fourier Transform. Note that we have to explicitly declare the Fourier Transform as an UDF (User Defined Function) so that it can act on SQL columns.

In [ ]:
def Fourier(x):
    x = np.array(x)
    # take real and imaginary components and getcomplex number  
    z = to_complex(x)
    
    power = np.abs(np.fft.fft(z))**2
    FS = fft_bandwidth
    norm = n_bins * FS * np.sqrt(2)
    normalized_power = power / norm
    power_shifted = np.fft.fftshift(normalized_power)
    
    power_shifted = power_shifted.tolist()
    
    return(power_shifted)

fft_udf = udf(Fourier, ArrayType(FloatType()))

In [ ]:
# Define your streaming DataFrame and apply transformations
streaming_df = inputDF.select('key', 'value')
streaming_df = streaming_df.withColumn('float', bytes_to_float32_udf(streaming_df['value']))
streaming_df = streaming_df.withColumn('fft', fft_udf(streaming_df['float']))


streaming_df = streaming_df.withColumn('file_num', extract_file_num_udf(col('key')))
streaming_df = streaming_df.withColumn('indexed_fft', indice_udf(streaming_df['fft'],streaming_df['file_num']) )
# Explode the 'indexed_fft' array to separate rows
exploded_df = streaming_df.select('key', explode('indexed_fft').alias('indexed_fft'))

We then extract the file number information from the *key* and, after some transformations, obtain a row for each bin with the following schema:

In [ ]:
root
 |-- key: binary (nullable = true)
 |-- indexed_fft: struct (nullable = true)
 |    |-- indice: integer (nullable = true)
 |    |-- x: float (nullable = true)

where *indexed_fft* is a structure containing two elements:
- *indice*, the combination of the file number and bin number
- *x*, the value fo the FFT in the specific bin for the specific file

Finally, we compute the mean and standard deviation of the FFTs for each bin after grouping by the FFT index.

In [ ]:
# Group by 'indexed_fft.indice' using dot notation
result_df = exploded_df.groupBy("indexed_fft.indice").agg(
    mean("indexed_fft.x").alias("mean_x"),
    stddev("indexed_fft.x").alias("stddev_x"),
    count("indexed_fft.x").alias("count_x")
)

#### 2.3.2 Output Message <a name="spark_output"></a>

The only step left is to produce a message to send Kafka in the *results* topic. This message is only sent if the whole file has been analyzed, i.e. if all 2731 slices have been used to compute the mean. 

The message will follow this schema:

In [ ]:
root
 |-- data: struct (nullable = false)
 |    |-- indice: integer (nullable = true)
 |    |-- mean_x: double (nullable = true)
 |    |-- stddev_x: double (nullable = true)
 |    |-- count_x: integer (nullable = true)

and be sent as Json file.

In [ ]:
# Send single json
result_json_df = result_df.where(col('count_x')==2731) \
    .select(struct("indice", "mean_x", "stddev_x","count_x").alias("data"))

def send_to_kafka(batch_df, batch_id):
    batch_json = batch_df.toJSON().collect()
    all_data_json = json.dumps([json.loads(row) for row in batch_json])
    
    producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)
    producer.send("results", value=all_data_json.encode("utf-8"))
    producer.close()

# Write the JSON data to Kafka as a single message
query = result_json_df.writeStream \
    .trigger(processingTime="20 seconds")\
    .outputMode("update") \
    .foreachBatch(send_to_kafka) \
    .start()

query.awaitTermination()

### 2.4 Live Plot and Monitoring <a name="live_plot"></a>

The only step left is to plot the results and monitor them in real time. To handle live plotting we will use Bokeh. 

We will show bpth the results for the latest batch sent to Kafka and the cumulative average of all the files processed up until now.

In [ ]:
cumulative_source.data['fft'] = (n_cum - 1)/n_cum*np.array(cumulative_source.data['fft']) + \
                                          1/n_cum*np.array(kafka_data['mean'])
cum_y = np.array(cumulative_source.data['fft'])
cum_sigma = 1/n_cum * np.sqrt(( n_cum -1 )**2 * cum_sigma**2 + sigma**2)

The final result looks like this:

<video width="861" height="505" 
       src="./Images\liveplot.webm"  
       controls>
</video>

## 3. Performance Tests <a name="test"></a>

### 3.2. Kafka <a name="test_kafka"></a>


In [10]:
import numpy as np
no_send = [0.19263148307800293,
           0.17882680892944336,
           0.19075679779052734,
           0.2853243350982666,
           0.22602128982543945,
           0.40697741508483887,
           0.22795748710632324,
           0.2530984878540039,
           0.23430776596069336,
           0.2378981113433838,
           0.23514914512634277,
           0.4101986885070801,
           0.2837076187133789,
           0.30994105339050293,
           0.3531005382537842,
           0.23813819885253906]

send_1 = [16.783443927764893,
          15.279634714126587,
          15.355290651321411,
          15.585991859436035,
          16.86496877670288,
          16.582424640655518,
          15.995649099349976,
          17.10170030593872,
          17.338494777679443,
          17.24843192100525,
          17.43307375907898,
          17.3827862739563,
          16.157559394836426,
          16.025066375732422,
          17.051117658615112,
          16.47578740119934]

send_10 = [15.814674854278564,
           15.83025074005127,
           15.502155065536499,
           15.249828577041626,
           15.124246835708618,
           14.991983413696289,
           15.013553857803345,
           14.93463397026062,
           16.170247793197632,
           16.067533493041992,
           15.825260639190674,
           15.469764709472656,
           15.535020589828491,
           15.316730976104736,
           15.207907915115356,
           16.054390907287598]

send_20 = [15.671099185943604,
           15.240856170654297,
           14.410763502120972,
           14.632994174957275,
           14.40667724609375,
           14.50796103477478,
           15.290517330169678,
           15.575354099273682,
           15.101133584976196,
           14.843635082244873,
           14.659634351730347,
           15.106323719024658,
           14.971516609191895,
           16.206547021865845,
           15.888291120529175,
           14.930995225906372]

send_30 = [15.126108884811401,
           14.870552062988281,
           14.777660131454468,
           14.550386905670166,
           14.341215372085571,
           14.874190092086792,
           15.806496143341064,
           15.014472961425781,
           14.799524307250977,
           15.306189060211182,
           15.480255603790283,
           15.310795783996582,
           14.410078048706055,
           15.332370281219482,
           14.388092756271362,
           15.743694305419922]

send_40 = [15.711830377578735,
           15.316266536712646,
           16.28687620162964,
           16.40558958053589,
           15.795770406723022,
           15.743608474731445,
           14.552511215209961,
           15.992870807647705,
           15.652602195739746,
           15.885672092437744,
           15.579556226730347,
           15.706321001052856,
           15.725404262542725,
           15.670507907867432,
           15.145171642303467,
           14.695325136184692]

times = [no_send, send_1, send_10, send_20, send_30, send_40]
for t in times:
    t = np.array(t)
    m = np.mean(t)
    s = np.std(t)
    print(f"Mean = {m:.2f} +- {s:.2f}")

Mean = 0.27 +- 0.07
Mean = 16.54 +- 0.71
Mean = 15.51 +- 0.40
Mean = 15.09 +- 0.52
Mean = 15.01 +- 0.45
Mean = 15.62 +- 0.48
